In [1]:
import os
os.listdir("../input/imet-weights2/")

['se_resnet152_288_v2020_oof6124.pt',
 'se_resnext101_32x4d_320_v2042_oof6179.pt',
 'densenet161_320_v42_oof6079.pt',
 'se_resnext101_32x4d_320_v3042_oof6159.pt',
 'densenet201_320_v42_oof6041_lb609.pt',
 'se_resnet101_288_v42_oof06083_lb617.pt',
 'se_resnext101_32x4d_320_v1042_oof6179.pt',
 'resnet101_288_v42_oof6083_lb616.pt',
 'se_resnext101_32x4d_320_v42_oof6152_lb621.pt',
 'se_resnet152_288_v42_oof6094_lb619.pt',
 'se_resnext101_32x4d_288_v3_lb623.pt']

- ResNet101 (288x288; 40fold)
  - v42 (LB 0.616): valid_f2_th_0.10: 0.6076048017265678
- SEResNet101 (288x288; 10fold)
  - v42 (LB 0.617): valid_f2_th_0.10: 0.60830836837079
- SEResNet152 (288x288; 10fold)
  - v42 (LB 0.619): valid_f2_th_0.10: 0.6094215115199988
  - v2020: valid_f2_th_0.09: 0.6124572071297274
- SEResNeXt101_32x4d (288x288; 10fold)
  - v3 (LB 0.623): valid_f2_th_0.08
- DenseNet201 (320x320; 10fold)
  - v42 (LB 0.609): valid_f2_th_0.10: 0.6041637785446881
- DenseNet161 (320x320; 10fold)
  - v42: valid_f2_th_0.11: 0.6079369147281679
- SEResNeXt101_32x4d (320x320; 10fold)
  - v42 (LB 0.621): valid_f2_th_0.09: 0.6152833096941255
  - v1024: valid_f2_th_0.08: 0.617966264921610
  - v2024: valid_f2 th_0.08: 0.6179158801859196
  - v3024: valid_f2_th_0.09: 0.6159644164340591

In [2]:
import gc
from functools import partial

import cv2
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.models as M
from torchvision.transforms import Compose, Normalize, RandomCrop, RandomResizedCrop, RandomHorizontalFlip, ToTensor

In [3]:
N_CLASSES = 1103

Use below models
- ResNet101
  - v42: 0.10
- SEResNet101
  - v42: 0.10
- SEResNet152
  - v2020: 0.09
- SEResNeXt101_32x4d (288x288)
  - v3: 0.08
- DenseNet161
  - v42: 0.11
- SEResNeXt101_32x4d (320x320)
  - v1024: 0.08
  - v2024: 0.08
  - v3024: 0.09

In [4]:
num_models = 8

In [5]:
threshold = (0.10+0.10+0.09+0.08+0.11+0.08+0.08+0.09) / num_models
threshold

0.09125

In [6]:
def load_image(path, item):
    image = cv2.imread(f'{path}/{item}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return Image.fromarray(image)

In [7]:
class TTADataset(Dataset):
    def __init__(self, root, ids, size=288, tta=4):
        self.root = root
        self.ids = ids
        
        if size == 288:
            self.transform = Compose([
                RandomCrop(288),
                RandomHorizontalFlip(),
                ToTensor(),
                Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            ])
        elif size == 320:
            self.transform = Compose([
                RandomResizedCrop(320, scale=(0.6, 1.0), ratio=(0.5625, 1.77777777)),  # 16:9/9:16
                RandomHorizontalFlip(),
                ToTensor(),
                Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            ])
            
        self.tta = tta

    def __len__(self):
        return len(self.ids) * self.tta

    def __getitem__(self, idx):
        item_id = self.ids[idx % len(self.ids)]
        image = load_image(self.root, item_id)
        image = self.transform(image)
        return image, item_id

## Model definitions

In [8]:
import math
from collections import OrderedDict

class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1,
                             padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1,
                             padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x


class Bottleneck(nn.Module):
    """
    Base class for bottlenecks that implements `forward()` method.
    """
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out


class SEBottleneck(Bottleneck):
    """
    Bottleneck for SENet154.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)
        self.conv2 = nn.Conv2d(planes * 2, planes * 4, kernel_size=3,
                               stride=stride, padding=1, groups=groups,
                               bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 4)
        self.conv3 = nn.Conv2d(planes * 4, planes * 4, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNetBottleneck(Bottleneck):
    """
    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe
    implementation and uses `stride=stride` in `conv1` and not in `conv2`
    (the latter is used in the torchvision implementation of ResNet).
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False,
                               stride=stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1,
                               groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNeXtBottleneck(Bottleneck):
    """
    ResNeXt bottleneck type C with a Squeeze-and-Excitation module.
    """
    expansion = 4

    def __init__(self, inplanes, planes, groups, reduction, stride=1,
                 downsample=None, base_width=4):
        super(SEResNeXtBottleneck, self).__init__()
        width = math.floor(planes * (base_width / 64)) * groups
        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False,
                               stride=1)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride,
                               padding=1, groups=groups, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SENet(nn.Module):

    def __init__(self, block, layers, groups, reduction, dropout_p=0.2,
                 inplanes=128, input_3x3=True, downsample_kernel_size=3,
                 downsample_padding=1, num_classes=1000):
        """
        Parameters
        ----------
        block (nn.Module): Bottleneck class.
            - For SENet154: SEBottleneck
            - For SE-ResNet models: SEResNetBottleneck
            - For SE-ResNeXt models:  SEResNeXtBottleneck
        layers (list of ints): Number of residual blocks for 4 layers of the
            network (layer1...layer4).
        groups (int): Number of groups for the 3x3 convolution in each
            bottleneck block.
            - For SENet154: 64
            - For SE-ResNet models: 1
            - For SE-ResNeXt models:  32
        reduction (int): Reduction ratio for Squeeze-and-Excitation modules.
            - For all models: 16
        dropout_p (float or None): Drop probability for the Dropout layer.
            If `None` the Dropout layer is not used.
            - For SENet154: 0.2
            - For SE-ResNet models: None
            - For SE-ResNeXt models: None
        inplanes (int):  Number of input channels for layer1.
            - For SENet154: 128
            - For SE-ResNet models: 64
            - For SE-ResNeXt models: 64
        input_3x3 (bool): If `True`, use three 3x3 convolutions instead of
            a single 7x7 convolution in layer0.
            - For SENet154: True
            - For SE-ResNet models: False
            - For SE-ResNeXt models: False
        downsample_kernel_size (int): Kernel size for downsampling convolutions
            in layer2, layer3 and layer4.
            - For SENet154: 3
            - For SE-ResNet models: 1
            - For SE-ResNeXt models: 1
        downsample_padding (int): Padding for downsampling convolutions in
            layer2, layer3 and layer4.
            - For SENet154: 1
            - For SE-ResNet models: 0
            - For SE-ResNeXt models: 0
        num_classes (int): Number of outputs in `last_linear` layer.
            - For all models: 1000
        """
        super(SENet, self).__init__()
        self.inplanes = inplanes
        if input_3x3:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,
                                    bias=False)),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                ('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                ('conv3', nn.Conv2d(64, inplanes, 3, stride=1, padding=1,
                                    bias=False)),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                ('conv1', nn.Conv2d(3, inplanes, kernel_size=7, stride=2,
                                    padding=3, bias=False)),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        # To preserve compatibility with Caffe weights `ceil_mode=True`
        # is used instead of `padding=1`.
        layer0_modules.append(('pool', nn.MaxPool2d(3, stride=2,
                                                    ceil_mode=True)))
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0
        )
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.avg_pool = nn.AvgPool2d(7, stride=1)
        self.dropout = nn.Dropout(dropout_p) if dropout_p is not None else None
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, groups, reduction, stride=1,
                    downsample_kernel_size=1, downsample_padding=0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=downsample_kernel_size, stride=stride,
                          padding=downsample_padding, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, groups, reduction, stride,
                            downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)

    def features(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, x):
        x = self.avg_pool(x)
        if self.dropout is not None:
            x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, x):
        x = self.features(x)
        x = self.logits(x)
        return x

In [9]:
class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])


class ResNet(nn.Module):
    def __init__(self, weights_path, net_cls=M.resnet101):
        super().__init__()
        self.net = net_cls()
        self.net.avgpool = AvgPool()
        self.net.fc = nn.Linear(self.net.fc.in_features, N_CLASSES)
        self.load_state_dict(torch.load(weights_path)['model'])

    def fresh_params(self):
        return self.net.fc.parameters()

    def forward(self, x):
        return self.net(x)


class DenseNet(nn.Module):
    def __init__(self, weights_path, net_cls=M.densenet121):
        super().__init__()
        self.net = net_cls()
        self.avg_pool = AvgPool()
        self.net.classifier = nn.Linear(self.net.classifier.in_features, N_CLASSES)
        self.load_state_dict(torch.load(weights_path)['model'])

    def fresh_params(self):
        return self.net.classifier.parameters()

    def forward(self, x):
        out = self.net.features(x)
        out = F.relu(out, inplace=True)
        out = self.avg_pool(out).view(out.size(0), -1)
        out = self.net.classifier(out)
        return out
    
class InceptionNet(nn.Module):
    def __init__(self, weights_path, net_cls=M.inception_v3):
        super().__init__()
        self.net = net_cls()
        self.net.fc = nn.Linear(self.net.fc.in_features, N_CLASSES)
        self.net.AuxLogits.fc = nn.Linear(self.net.AuxLogits.fc.in_features, N_CLASSES)
        self.load_state_dict(torch.load(weights_path)['model'])

    def fresh_params(self):
        return self.net.fc.parameters()

    def forward(self, x):
        return self.net(x)
    
#resnet18 = partial(ResNet, net_cls=M.resnet18)
#resnet34 = partial(ResNet, net_cls=M.resnet34)
#resnet50 = partial(ResNet, net_cls=M.resnet50)
resnet101 = partial(ResNet, net_cls=M.resnet101)
#resnet152 = partial(ResNet, net_cls=M.resnet152)

#densenet121 = partial(DenseNet, net_cls=M.densenet121)
#densenet169 = partial(DenseNet, net_cls=M.densenet169)
densenet201 = partial(DenseNet, net_cls=M.densenet201)
densenet161 = partial(DenseNet, net_cls=M.densenet161)

In [10]:
class SEResNet50(nn.Module):
    def __init__(self, weights_path, pretrained=False, dropout=False, net_cls=None):
        super().__init__()
        self.net = SENet(SEResNetBottleneck, [3,4,6,3], groups=1, reduction=16,
                         dropout_p=None, inplanes=64, input_3x3=False,
                         downsample_kernel_size=1, downsample_padding=0,
                         num_classes=N_CLASSES)
        self.net.avg_pool = AvgPool()
        self.load_state_dict(torch.load(weights_path)['model'])
        #self.net.last_linear = nn.Linear(self.net.last_linear.in_features, num_classes)

    def fresh_params(self):
        return self.net.last_linear.parameters()

    def forward(self, x):
        return self.net(x)


class SEResNet101(nn.Module):
    def __init__(self, weights_path, pretrained=False, dropout=False, net_cls=None):
        super().__init__()
        self.net = SENet(SEResNetBottleneck, [3,4,23,3], groups=1, reduction=16,
                         dropout_p=None, inplanes=64, input_3x3=False,
                         downsample_kernel_size=1, downsample_padding=0,
                         num_classes=N_CLASSES)
        self.net.avg_pool = AvgPool()
        self.load_state_dict(torch.load(weights_path)['model'])
        #self.net.last_linear = nn.Linear(self.net.last_linear.in_features, num_classes)

    def fresh_params(self):
        return self.net.last_linear.parameters()

    def forward(self, x):
        return self.net(x)


class SEResNet152(nn.Module):
    def __init__(self, weights_path, pretrained=False, dropout=False, net_cls=None):
        super().__init__()
        self.net = SENet(SEResNetBottleneck, [3,8,36,3], groups=1, reduction=16,
                         dropout_p=None, inplanes=64, input_3x3=False,
                         downsample_kernel_size=1, downsample_padding=0,
                         num_classes=N_CLASSES)
        self.net.avg_pool = AvgPool()
        self.load_state_dict(torch.load(weights_path)['model'])
        #self.net.last_linear = nn.Linear(self.net.last_linear.in_features, num_classes)

    def fresh_params(self):
        return self.net.last_linear.parameters()

    def forward(self, x):
        return self.net(x)


class SEResNeXt50_32x4d(nn.Module):
    def __init__(self, weights_path, pretrained=False, dropout=False, net_cls=None):
        super().__init__()
        self.net = SENet(SEResNeXtBottleneck, [3,4,6,3], groups=32, reduction=16,
                         dropout_p=None, inplanes=64, input_3x3=False,
                         downsample_kernel_size=1, downsample_padding=0,
                         num_classes=N_CLASSES)
        self.net.avg_pool = AvgPool()
        self.load_state_dict(torch.load(weights_path)['model'])
        #self.net.last_linear = nn.Linear(self.net.last_linear.in_features, num_classes)

    def fresh_params(self):
        return self.net.last_linear.parameters()

    def forward(self, x):
        return self.net(x)


class SEResNeXt101_32x4d(nn.Module):
    def __init__(self, weights_path, pretrained=False, dropout=False, net_cls=None):
        super().__init__()
        self.net = SENet(SEResNeXtBottleneck, [3,4,23,3], groups=32, reduction=16,
                         dropout_p=None, inplanes=64, input_3x3=False,
                         downsample_kernel_size=1, downsample_padding=0,
                         num_classes=N_CLASSES)
        self.net.avg_pool = AvgPool()
        self.load_state_dict(torch.load(weights_path)['model'])
        #self.net.last_linear = nn.Linear(self.net.last_linear.in_features, num_classes)

    def fresh_params(self):
        return self.net.last_linear.parameters()

    def forward(self, x):
        return self.net(x)


se_resnet50 = partial(SEResNet50, net_cls=None)
se_resnet101 = partial(SEResNet101, net_cls=None)
se_resnet152 = partial(SEResNet152, net_cls=None)
se_resnext50_32x4d = partial(SEResNeXt50_32x4d, net_cls=None)
se_resnext101_32x4d = partial(SEResNeXt101_32x4d, net_cls=None)

In [11]:
sub = pd.read_csv('../input/imet-2019-fgvc6/sample_submission.csv')
sub.head()

,id,attribute_ids
0,10023b2cc4ed5f68,0 1 2
1,100fbe75ed8fd887,0 1 2
2,101b627524a04f19,0 1 2
3,10234480c41284c6,0 1 2
4,1023b0e2636dcea8,0 1 2


## Predictions

In [12]:
final_preds = np.zeros((len(sub), N_CLASSES))
final_preds.shape

(7443, 1103)

### 288x288

In [13]:
test_dataset = TTADataset('../input/imet-2019-fgvc6/test', sub.id.values, size=288, tta=3)

#### SEResNet101

In [14]:
model = se_resnet101('../input/imet-weights2/se_resnet101_288_v42_oof06083_lb617.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

#### SEResNet152

In [15]:
model = se_resnet152('../input/imet-weights2/se_resnet152_288_v2020_oof6124.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

#### ResNet101

In [16]:
model = resnet101('../input/imet-weights2/resnet101_288_v42_oof6083_lb616.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

#### SEResNeXt101 32x4d (288)

In [17]:
model = se_resnext101_32x4d('../input/imet-weights2/se_resnext101_32x4d_288_v3_lb623.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

### 320x320

In [18]:
test_dataset = TTADataset('../input/imet-2019-fgvc6/test', sub.id.values, size=320, tta=6)

#### DenseNet161

In [19]:
model = densenet161('../input/imet-weights2/densenet161_320_v42_oof6079.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

#### SEResNeXt101 32x4d

In [20]:
model = se_resnext101_32x4d('../input/imet-weights2/se_resnext101_32x4d_320_v1042_oof6179.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

In [21]:
model = se_resnext101_32x4d('../input/imet-weights2/se_resnext101_32x4d_320_v2042_oof6179.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

In [22]:
model = se_resnext101_32x4d('../input/imet-weights2/se_resnext101_32x4d_320_v3042_oof6159.pt')
model.cuda()
model.eval()

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

all_outputs, all_ids = [], []

for imgs, ids in test_loader:
    preds = torch.sigmoid(model(imgs.cuda()).detach())
    all_outputs.append(preds.cpu().numpy())
    all_ids.extend(ids)
    
test_preds = pd.DataFrame(data=np.concatenate(all_outputs),
                          index=all_ids,
                          columns=map(str, range(N_CLASSES)))
test_preds = test_preds.groupby(level=0).mean()

final_preds += test_preds.values
del model, test_preds
gc.collect();

## Create submission

In [23]:
tmp = (final_preds / num_models) > threshold
tmp

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [24]:
for i, row in enumerate(tmp):
    ids = np.nonzero(row)[0]
    # no men but acotors
    if (404 in ids) and (813 not in ids):
        tmp[i][404] = 0
    # max 4 culture labels
    #k = 4
    #if tmp[i][:398].sum()>k:
    #    ids = np.argpartition(-final_preds[i][:398], k)[:k]
    #    tmp[i][:398] = 0
    #    tmp[i][ids] = 1

In [25]:
for i, row in enumerate(tmp):
    ids = np.nonzero(row)[0]
    sub.iloc[i].attribute_ids = ' '.join([str(x) for x in ids])

In [26]:
sub.to_csv('submission.csv', index=False)

In [27]:
sub.head()

,id,attribute_ids
0,10023b2cc4ed5f68,223 289 343 344 369 587 766 1059
1,100fbe75ed8fd887,93 231 1039
2,101b627524a04f19,79 420 784
3,10234480c41284c6,13 147 480 483 725 738 776 813 830 1046
4,1023b0e2636dcea8,147 322 584 737 738 776 813 954 1046 1092
